## 定义模型2 alexnet

In [2]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, transforms
from torch.utils.data import DataLoader
import matplotlib.pyplot as plt
import torch.nn.functional as F
from torch.utils.data import DataLoader, TensorDataset
import pandas as pd

%matplotlib qt

In [7]:
model_accuracies = {
    'alexnet': []
}
# 定义批量大小
batch_size = 100
# 数据增强和预处理
transform2 = transforms.Compose([
    transforms.Resize((227, 227)),  ##图像大小转化，转化为各深度学习分类器要求的图像大小
    transforms.ToTensor(),  ## 将图像转换为PyTorch的张量格式，并将像素值从0-255缩放到0-1之间
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])  ## 标准化图像，使每个通道的像素值分布在[-1, 1]之间。这里，均值和标准差均为0.485, 0.456, 0.406。
])

In [8]:
# 加载数据集
data_dir = r'D:\深度学习练习\flower_photos\flower_photos'
train_dataset = datasets.ImageFolder(root=data_dir, transform=transform2)#加载数据集并使用相应的转换器转换
train_size = int(0.8 * len(train_dataset))
val_size = len(train_dataset) - train_size
train_dataset, val_dataset = torch.utils.data.random_split(train_dataset, [train_size, val_size])

train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)

In [11]:
class AlexNet(nn.Module):
    def __init__(self, num_classes=4):
        super(AlexNet, self).__init__()
        self.features = nn.Sequential(
            nn.Conv2d(3, 96, kernel_size=11, stride=4, padding=2),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=3, stride=2),
            nn.Conv2d(96, 256, kernel_size=5, padding=2),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=3, stride=2),
            nn.Conv2d(256, 384, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.Conv2d(384, 384, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.Conv2d(384, 256, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=3, stride=2),
        )
        self.classifier = nn.Sequential(
            nn.Dropout(),
            nn.Linear(256 * 6 * 6, 4096),
            nn.ReLU(inplace=True),
            nn.Dropout(),
            nn.Linear(4096, 4096),
            nn.ReLU(inplace=True),
            nn.Linear(4096, num_classes),
        )

    def forward(self, x):
        x = self.features(x)
        x = x.view(x.size(0), 256 * 6 * 6)
        x = self.classifier(x)
        return x
# 实例化模型
model = AlexNet(num_classes=5)
epochs = 40
learning_rate = 0.0001

In [12]:
from bokeh.models.widgets import inputs

# 假设 device 已经定义
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# 初始化模型、损失函数和优化器
model = AlexNet(num_classes=5).to(device)  # 确保模型在 GPU 上
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

# 训练模型

train_losses = []
for epoch in range(epochs):
    model.train()
    running_loss = 0.0
    for images, labels in train_loader:
        images, labels = images.to(device), labels.to(device)  # 将数据移动到 GPU
        optimizer.zero_grad()  # 清除梯度
        outputs = model(images)  # 前向传播
        loss = criterion(outputs, labels)  # 计算损失
        loss.backward()  # 反向传播
        optimizer.step()  # 更新参数
        running_loss += loss.item()
    
    epoch_loss = running_loss / len(train_loader)
    train_losses.append(epoch_loss)
    print(f"Epoch [{epoch+1}/{epochs}], Loss: {epoch_loss:.4f}")

    # 验证模型
    model.eval()
    correct = 0
    total = 0
    with torch.no_grad():
        for images, labels in val_loader:
            images, labels = images.to(device), labels.to(device)  # 将数据移动到 GPU
            outputs = model(images)  # 前向传播
            _, predicted = torch.max(outputs.data, 1)  # 获取预测类别
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
    
    accuracy = 100 * correct / total
    model_accuracies['alexnet'].append(accuracy)
    print(f"Accuracy of the model on the validation images: {accuracy:.2f}%")

Epoch [1/40], Loss: 1.4826
Accuracy of the model on the validation images: 32.70%
Epoch [2/40], Loss: 1.2169
Accuracy of the model on the validation images: 51.23%
Epoch [3/40], Loss: 1.1355
Accuracy of the model on the validation images: 55.99%
Epoch [4/40], Loss: 1.0452
Accuracy of the model on the validation images: 57.22%


KeyboardInterrupt: 

In [6]:

# 保存模型权重
torch.save(model.state_dict(), 'alexnet_weights.pth')
# 保存模型结构和权重
torch.save(model, 'alexnet_model.pth')
accuracies_list = model_accuracies['alexnet']

# 创建一个包含epoch编号和对应准确率的字典
data = {'Epoch': list(range(1, len(accuracies_list) + 1)),
        'Accuracy': accuracies_list}

# 将字典转换为DataFrame
df = pd.DataFrame(data)

# 现在你可以将DataFrame保存为Excel文件
df.to_excel('alexnet_accuracies.xlsx', index=False)
# 清理内存
del model
del inputs
del labels
torch.cuda.empty_cache()

In [7]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")
# 实例化模型
model = AlexNet(num_classes=5).to(device)

# 加载模型权重
model.load_state_dict(torch.load('alexnet_weights.pth'))

# 设置为评估模式
model.eval()

Using device: cuda


AlexNet(
  (features): Sequential(
    (0): Conv2d(3, 96, kernel_size=(11, 11), stride=(4, 4), padding=(2, 2))
    (1): ReLU(inplace=True)
    (2): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
    (3): Conv2d(96, 256, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
    (4): ReLU(inplace=True)
    (5): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
    (6): Conv2d(256, 384, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (7): ReLU(inplace=True)
    (8): Conv2d(384, 384, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (9): ReLU(inplace=True)
    (10): Conv2d(384, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): ReLU(inplace=True)
    (12): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (classifier): Sequential(
    (0): Dropout(p=0.5, inplace=False)
    (1): Linear(in_features=9216, out_features=4096, bias=True)
    (2): ReLU(inplace=True)
    (3): Dropout(p=0.5, 

In [8]:
import torch
import csv
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from tqdm import tqdm

# 确保模型处于评估模式
model.eval()

# 初始化用于存储预测和真实标签的列表
all_preds = []
all_labels = []

# 不计算梯度进行预测
with torch.no_grad():
    for inputs, labels in tqdm(val_loader, desc="Evaluating"):
        # 将数据移动到GPU，如果可用
        inputs, labels = inputs.to(device), labels.to(device)

        # 前向传播
        outputs = model(inputs)

        # 获取预测概率最高的类别
        _, predicted = torch.max(outputs.data, 1)

        # 将预测结果和真实标签添加到列表中
        all_preds.extend(predicted.cpu().numpy())
        all_labels.extend(labels.cpu().numpy())

# 计算准确率、精确率、召回率和F1分数
accuracy = accuracy_score(all_labels, all_preds)
precision = precision_score(all_labels, all_preds, average='weighted', zero_division=1)
recall = recall_score(all_labels, all_preds, average='weighted', zero_division=1)
f1 = f1_score(all_labels, all_preds, average='weighted', zero_division=1)

# 将结果保存到CSV文件
results = {
    'Accuracy': accuracy,
    'Precision': precision,
    'Recall': recall,
    'F1 Score': f1
}

# 写入CSV文件
with open('AlexNet评估.csv', mode='w', newline='') as file:
    writer = csv.writer(file)
    writer.writerow(['Metric', 'Value'])
    for metric, value in results.items():
        writer.writerow([metric, value])

print("Results have been saved to AlexNet评估.csv")

Evaluating: 100%|██████████| 4/4 [00:02<00:00,  1.68it/s]

Results have been saved to AlexNet评估.csv
